In [ ]:
import json
import subprocess
from youtubesearchpython import VideosSearch
from typing import List, Dict

In [ ]:
def duration_to_int(dur):
    dd = 0
    tt = 0  # power of 60
    for ii in dur.split(":")[::-1]:
        dd += (60 ** tt) * int(ii)
        tt += 1
    return dd

## Raag Search

In [ ]:
vocal_artists = []
instr_artists = []

In [ ]:
def query_builder(raag) -> List[str]:
    links = []
    
    # Bada Khyal
    b_res = VideosSearch(f"{raag} Khyaal", limit = 3).result()["result"]
    b_res = sorted(b_res, key=lambda x: duration_to_int(x["duration"]))[::-1]
    # constrain duration between 30-90 minutes
    for br in b_res:
        if 1800 <= duration_to_int(br["duration"]) <= 5400:
            print(f"adding {br['title']}")
            links.append(br["link"])
    
    # Chhota Khyal
    c_res = VideosSearch(f"{raag} Vocal Drut", limit = 3).result()["result"]
    c_res = sorted(c_res, key=lambda x: duration_to_int(x["duration"]))[::-1]
    # constrain duration between 7-25 minutes
    for cr in c_res:
        if 420 <= duration_to_int(cr["duration"]) <= 1500:
            print(f"adding {cr['title']}")
            links.append(cr["link"])
    
    # Instrumental
    i_res = VideosSearch(f"{raag} Instrumental", limit = 3).result()["result"]
    i_res = sorted(i_res, key=lambda x: duration_to_int(x["duration"]))[::-1]
    # constrain duration between 10-45 minutes
    for ir in i_res:
        if 600 <= duration_to_int(ir["duration"]) <= 2700:
            print(f"adding {ir['title']}")
            links.append(ir["link"])
    
    return links

## Gather Links

In [ ]:
bl = query_builder("Bhoop")

In [ ]:
bl2 = query_builder("Bhoopali")

In [ ]:
mbl = query_builder("भूपाली")

## Get Audios

In [ ]:
urls = []

In [ ]:
for i in urls:
    command = "yt-dlp" + "https://www.youtube.com/watch?v=n0IT7_n4n_U" + "-f 'ba' -x --audio-format 'mp3' -P raagdata/"
    result = subprocess.run(command, shell=True, capture_output=True)
    print(result.stdout)
    print(result.stderr)

## Get Chunks

In [ ]:
"""
1. Heuristic to sample within spans
2. Confirm that this is not speech
3. Good file naming
4. Carry over some metadata from the youtube search..
    This will be very important for credit attribution!!
"""